In [159]:
import pandas as pd
import numpy as np
import os
import sys

import re
from sympy.logic.boolalg import to_cnf
from sympy.parsing.sympy_parser import parse_expr
from sympy import Symbol
import sympy
import z3

### Read the data

In [5]:
current_path = os.getcwd()
day = int(current_path.split('day')[1])

fn = 'day' + str(day) + '.txt'

file_content = open(fn).read().split('\n')

test_fn = 'day' + str(day) + 'test.txt'

test_file_content = open(test_fn).read().split('\n')

### Part 1

In [43]:

def part_accepted(workflow_dict, workflow_name, part):
    if workflow_name == 'A':
        return True
    elif workflow_name == 'R':
        return False

    x, m, a, s = part
    workflow = workflow_dict[workflow_name]
    for workflow_step in workflow[:-1]:
        if workflow_step[0] == 'x':	
            if workflow_step[1] == '<':
                if x < int(workflow_step.split(':')[0][2:]):
                    return part_accepted(workflow_dict, workflow_step.split(':')[1], part)
                else:
                    continue
            elif workflow_step[1] == '>':
                if x > int(workflow_step.split(':')[0][2:]):
                    return part_accepted(workflow_dict, workflow_step.split(':')[1], part)
                else:
                    continue
        if workflow_step[0] == 'm':
            if workflow_step[1] == '<':
                if m < int(workflow_step.split(':')[0][2:]):
                    return part_accepted(workflow_dict, workflow_step.split(':')[1], part)
                else:
                    continue
            elif workflow_step[1] == '>':
                if m > int(workflow_step.split(':')[0][2:]):
                    return part_accepted(workflow_dict, workflow_step.split(':')[1], part)
                else:
                    continue
        if workflow_step[0] == 'a':
            if workflow_step[1] == '<':
                if a < int(workflow_step.split(':')[0][2:]):
                    return part_accepted(workflow_dict, workflow_step.split(':')[1], part)
                else:
                    continue
            elif workflow_step[1] == '>':
                if a > int(workflow_step.split(':')[0][2:]):
                    return part_accepted(workflow_dict, workflow_step.split(':')[1], part)
                else:
                    continue
        if workflow_step[0] == 's':
            if workflow_step[1] == '<':
                if s < int(workflow_step.split(':')[0][2:]):
                    return part_accepted(workflow_dict, workflow_step.split(':')[1], part)
                else:
                    continue
            elif workflow_step[1] == '>':
                if s > int(workflow_step.split(':')[0][2:]):
                    return part_accepted(workflow_dict, workflow_step.split(':')[1], part)
                else:
                    continue
    return part_accepted(workflow_dict, workflow[-1], part)

In [48]:
working_file = file_content

empty_line_index = working_file.index('')

workflows = working_file[:empty_line_index]
part_ratings = working_file[empty_line_index+1:]

workflow_dict = {}
for workflow in workflows:
    workflow_dict[workflow.split('{')[0]] = workflow.split('{')[1][:-1].split(',')

part_ratings = [x.replace('{','').replace('}','').split(',') for x in part_ratings]
part_ratings = [[int(x[2:]), int(m[2:]), int(a[2:]), int(s[2:])] for x, m, a, s in part_ratings]

sum_of_accepted = 0
for part in part_ratings:
    if part_accepted(workflow_dict, 'in', part):
        sum_of_accepted += sum(part)
print(sum_of_accepted)

456651


### Part 2

In [187]:
import re
ll = [x for x in open('day19.txt').read().strip().split('\n\n')]
workflow, parts = ll

def ints(s):
	return list(map(int, re.findall(r'\d+', s)))

parts = [ints(l) for l in parts.split("\n")]
workflow = {l.split("{")[0]: l.split("{")[1][:-1] for l in workflow.split("\n")}

def both(ch, gt, val, ranges):
	ch = 'xmas'.index(ch)
	ranges2 = []
	for rng in ranges:
		rng = list(rng)
		lo, hi = rng[ch]
		if gt:
			lo = max(lo, val + 1)
		else:
			hi = min(hi, val - 1)
		if lo > hi:
			continue
		rng[ch] = (lo, hi)
		ranges2.append(tuple(rng))
	return ranges2


def acceptance_ranges_outer(work):
	return acceptance_ranges_inner(workflow[work].split(","))

def acceptance_ranges_inner(w):
	it = w[0]
	if it == "R":
		return []
	if it == "A":
		return [((1, 4000), (1, 4000), (1, 4000), (1, 4000))]
	if ":" not in it:
		return acceptance_ranges_outer(it)
	cond = it.split(":")[0]
	gt = ">" in cond
	ch = cond[0]
	val = int(cond[2:])
	val_inverted = val + 1 if gt else val - 1
	if_cond_is_true = both(ch, gt, val, acceptance_ranges_inner([it.split(":")[1]]))
	if_cond_is_false = both(ch, not gt, val_inverted, acceptance_ranges_inner(w[1:]))
	return if_cond_is_true + if_cond_is_false

p2 = 0
for rng in acceptance_ranges_outer('in'):
	v = 1
	for lo, hi in rng:
		v *= hi - lo + 1
	p2 += v
print(p2)

131899818301477
